<a href="https://colab.research.google.com/github/lbrogna/football_data_for_ml/blob/main/processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook was written to process data from football-data.co.uk into a more usable format for machine learning applications.

Here is the catalogue of features contained in this dataset off the shelf: https://www.football-data.co.uk/notes.txt

Up to date data can be downloaded from this URL: https://www.football-data.co.uk/downloadm.php. 

Data can be downloaded by season. When downloaded from the archives, files will be named using the same naming convention as seen in the below code cell. To use this notebook, download the desired years and upload them to your Google Drive.

Alternatively, one can just use the example data provided in the 'example_data' included in the GitHub

NOTE: the code must be tweaked if using years before 2019-2020, as more data began being collected then, so the data from prior years has slightly different columns.

The dataset comes to us in the form of match by match stats.

Along with cleaning, the data will be augmented in this notebook in the following ways:

1. Add to every match row historical statistics from the last n matches played by both teams.
2. Add to every match row columns which reflect each team's position in their league at the time of the match (like a league table).
3. Add to every match row time delta columns representing the amount of days since they played their last n matches.

An example of the the already processed data can be found in the 'example_data' folder if you want to skip to the 'demo' notebook.

In [27]:
import pandas as pd
import numpy as np
from google.colab import drive

In [28]:
import warnings
warnings.filterwarnings('ignore')

Run this code with your own modifications if you wish to data that you have downloaded to your Google drive, otherwise, run the code cell below it to fetch the example data.

In [29]:
drive.mount('/content/drive')

# change these paths to represent your data
file_paths = ['/content/drive/MyDrive/improved_odds/all-euro-data-2022-2023.xlsx', 
              '/content/drive/MyDrive/improved_odds/all-euro-data-2021-2022.xlsx', 
              '/content/drive/MyDrive/improved_odds/all-euro-data-2020-2021.xlsx', 
              '/content/drive/MyDrive/improved_odds/all-euro-data-2019-2020.xlsx']


# Create an empty list to store dataframes
df_list = []

#initialize value to be added as a season column
season_num = '1'

# Loop through each file path in the list
for file_path in file_paths:
    # Read the Excel file
    excel_file = pd.read_excel(file_path, sheet_name=None)

    # Loop through each sheet in the Excel file
    for sheet_name, df in excel_file.items():
        # Add a new column to the dataframe to represent the sheet name
        df['season'] = season_num
        # Append the dataframe to the list
        df_list.append(df)
    
    season_num = str(int(season_num) + 1)

# Concatenate all dataframes into a master dataframe
master_df = pd.concat(df_list, ignore_index=True)

master_df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,season
0,E0,2022-08-05,20:00:00,Crystal Palace,Arsenal,0,2,A,0.0,1.0,...,0.50,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,1
1,E0,2022-08-06,12:30:00,Fulham,Liverpool,2,2,D,1.0,0.0,...,1.75,1.90,2.03,1.91,2.02,2.01,2.06,1.89,1.99,1
2,E0,2022-08-06,15:00:00,Bournemouth,Aston Villa,2,0,H,1.0,0.0,...,0.50,1.93,2.00,1.93,2.00,1.94,2.04,1.88,2.00,1
3,E0,2022-08-06,15:00:00,Leeds,Wolves,2,1,H,1.0,1.0,...,-0.25,2.08,1.85,2.10,1.84,2.14,1.87,2.08,1.81,1
4,E0,2022-08-06,15:00:00,Newcastle,Nott'm Forest,2,0,H,0.0,0.0,...,-1.00,1.97,1.96,1.99,1.93,2.19,1.97,2.03,1.86,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28161,G1,2020-07-18,17:15:00,Larisa,Xanthi,0,0,D,0.0,0.0,...,0.00,2.05,1.80,2.07,1.78,2.14,1.83,2.06,1.78,4
28162,G1,2020-07-18,17:15:00,Panetolikos,Volos NFC,1,0,H,1.0,0.0,...,-1.00,1.83,2.02,1.85,2.00,1.88,2.07,1.82,2.01,4
28163,G1,2020-07-19,18:00:00,Olympiakos,AEK,3,0,H,2.0,0.0,...,-0.75,1.95,1.90,1.93,1.93,2.00,1.97,1.92,1.91,4
28164,G1,2020-07-19,18:00:00,Panathinaikos,OFI Crete,3,2,H,3.0,2.0,...,-1.00,1.93,1.93,1.90,1.95,1.99,2.00,1.90,1.92,4


Below are functions that create contemporary league tables for every match, and derive columns from those league tables to add to every row. 

In [30]:
def update_league_table(table, home_team, away_team, home_score, away_score):
    """
    Update the league table with the result of a match.

    Args:
        table (pandas.DataFrame): The league table to update.
        home_team (str): The name of the home team.
        away_team (str): The name of the away team.
        home_score (int): The number of goals scored by the home team.
        away_score (int): The number of goals scored by the away team.

    Returns:
        None
    """
    # Update the goal differential
    table.at[home_team, 'GF'] += home_score
    table.at[home_team, 'GA'] += away_score
    table.at[home_team, 'GD'] = table.at[home_team, 'GF'] - table.at[home_team, 'GA']
    
    table.at[away_team, 'GF'] += away_score
    table.at[away_team, 'GA'] += home_score
    table.at[away_team, 'GD'] = table.at[away_team, 'GF'] - table.at[away_team, 'GA']

    # update the matches played
    table.at[home_team, 'MP'] += 1
    table.at[away_team, 'MP'] += 1

    # Update the points based on the result
    if home_score > away_score:
        table.at[home_team, 'W'] += 1
        table.at[home_team, 'P'] += 3
        table.at[away_team, 'L'] += 1
    elif home_score == away_score:
        table.at[home_team, 'D'] += 1
        table.at[home_team, 'P'] += 1
        table.at[away_team, 'D'] += 1
        table.at[away_team, 'P'] += 1
    else:
        table.at[home_team, 'L'] += 1
        table.at[away_team, 'W'] += 1
        table.at[away_team, 'P'] += 3


def create_empty_league_table(df, league, season):
    """
    Create an empty league table with columns for team names, matches played, wins, draws, losses,     
    goals for, goals against, goal differential, and points.

    Args:
        df (pandas.DataFrame): The dataframe containing the match data.
        league (str): The name of the league.
        season (str): The season of the league.

    Returns:
        pandas.DataFrame: An empty league table.
    """
    # Get unique team names for the given league and season
    teams = df.loc[(df['Div'] == league) & (df['season'] == season), ['HomeTeam', 'AwayTeam']].stack().unique()
    
    # Create an empty dataframe with columns for team names, matches played, wins, draws, losses,     
    # goals for, goals against, goal differential, and points
    table = pd.DataFrame(columns=['team', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'P'])
    table['team'] = teams
    table.set_index('team', inplace=True)
    table[['MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'P']] = 0
    
    return table


def add_league_table_columns(df, n_ranks):
    """
    Add columns derived from league table.

    Args:
        df (pandas.DataFrame): The dataframe containing the match data.
        ranks (int): The number of ranks in the resulting league table columns.

    """
    # Create an empty dictionary to hold league tables for each league/season combination
    league_tables = {}
      
    # Loop through each row in the dataframe in chronological order
    for _, row in df.sort_values('Date').iterrows():
        # Get the league and season for the current row
        league = row['Div']
        season = row['season']
        
        # Create a new league table if one doesn't exist for this league/season combination
        if (league, season) not in league_tables:
            league_tables[(league, season)] = create_empty_league_table(df, league, season)
        
        # game_count = len(df[(df['Div'] == league) & (df['season'] == season)])

        # Update the league table with the result of the current match
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        home_score = row['FTHG']
        away_score = row['FTAG']
        
        # Get the league table for the current league/season combination
        table = league_tables[(league, season)]
        
        # Calculate the completion percentage of the season so far
        # matches_per_team = game_count / len(table)
        current_match = row.name
        # season_completion = ((table.at[home_team, 'MP'] + table.at[away_team, 'MP']) / 2) / matches_per_team
        
        # Add the season completion percentage column to the dataframe
        # df.at[current_match, 'season_completion'] = season_completion
        
        table_positions = np.linspace(1/len(table), 1, n_ranks)
        position_list = list(range(n_ranks, 0, -1))

        # Add columns representing the absolute difference in points and goal difference
        # between the given team and all other ranks on the league table
        home_team_rank = table.index.get_loc(home_team) + 1  # Add 1 to start from rank 1 instead of 0
        away_team_rank = table.index.get_loc(away_team) + 1
        name_pos = 0
        for rank in table_positions:
            if rank == home_team_rank/len(table):
                df.at[current_match, f'H_rank_{position_list[name_pos]}_abs_diff_points'] = 0
                df.at[current_match, f'H_rank_{position_list[name_pos]}_diff_gd'] = 0
            else:
                rank_team = table.iloc[int(len(table) * rank) - 1].name  # Get the team name at the current rank
                df.at[current_match, f'H_rank_{position_list[name_pos]}_abs_diff_points'] = abs(table.at[home_team, 'P'] - table.at[rank_team, 'P'])
                df.at[current_match, f'H_rank_{position_list[name_pos]}_diff_gd'] = table.at[home_team, 'GD'] - table.at[rank_team, 'GD']
                
            if rank == away_team_rank/len(table):
                df.at[current_match, f'A_rank_{position_list[name_pos]}_abs_diff_points'] = 0
                df.at[current_match, f'A_rank_{position_list[name_pos]}_diff_gd'] = 0
            else:
                rank_team = table.iloc[int(len(table) * rank) - 1].name  # Get the team name at the current rank
                df.at[current_match, f'A_rank_{position_list[name_pos]}_abs_diff_points'] = abs(table.at[away_team, 'P'] - table.at[rank_team, 'P'])
                df.at[current_match, f'A_rank_{position_list[name_pos]}_diff_gd'] = table.at[away_team, 'GD'] - table.at[rank_team, 'GD']
            name_pos+=1
        update_league_table(league_tables[(league, season)], home_team, away_team, home_score, away_score)
    
    return df

I'm dropping the 'Referee' column. I'm assuming we don't have enough matches for each individual referee to learn anything meaningful. We only have their names, so the only option would be to one-hot encode, which I suspect would only stunt any machine learning model.

In [31]:
df = master_df.drop(['Referee'], axis=1)

Below, I change the 'Date' column to be a proper datetime data type. I also drop the 'Time' column because I'm not that interested in when during the day each game was played.

In [32]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.drop(columns='Time')

Now, I'll use the functions I coded above to add columns that represent position on the league table (or suitable proxy) for each team on a given match day. Not every league has the same amount of teams, so I opted to have 10 league table position columns, indicating distance in points and goal difference from 10 locations on the table (first place, last place, and 8 others in between). The number of these positions can be changed using the n_ranks parameter. 

I say suitable proxy above because not every league has the same rules regarding tiebreakers. Still, points and goal difference should be plenty to get a good idea of the strength of each team compared to the league that season, as well as to their opponent.

In [33]:
add_league_table_columns(df, n_ranks=10)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,A_rank_3_abs_diff_points,A_rank_3_diff_gd,H_rank_2_abs_diff_points,H_rank_2_diff_gd,A_rank_2_abs_diff_points,A_rank_2_diff_gd,H_rank_1_abs_diff_points,H_rank_1_diff_gd,A_rank_1_abs_diff_points,A_rank_1_diff_gd
0,E0,2022-08-05,Crystal Palace,Arsenal,0,2,A,0.0,1.0,A,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,E0,2022-08-06,Fulham,Liverpool,2,2,D,1.0,0.0,H,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E0,2022-08-06,Bournemouth,Aston Villa,2,0,H,1.0,0.0,H,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,E0,2022-08-06,Leeds,Wolves,2,1,H,1.0,1.0,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,E0,2022-08-06,Newcastle,Nott'm Forest,2,0,H,0.0,0.0,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28161,G1,2020-07-18,Larisa,Xanthi,0,0,D,0.0,0.0,D,...,37.0,-42.0,6.0,8.0,6.0,5.0,0.0,3.0,0.0,0.0
28162,G1,2020-07-18,Panetolikos,Volos NFC,1,0,H,1.0,0.0,H,...,41.0,-55.0,0.0,0.0,5.0,-7.0,9.0,-6.0,4.0,-13.0
28163,G1,2020-07-19,Olympiakos,AEK,3,0,H,2.0,0.0,H,...,4.0,1.0,59.0,73.0,40.0,48.0,52.0,68.0,33.0,43.0
28164,G1,2020-07-19,Panathinaikos,OFI Crete,3,2,H,3.0,2.0,H,...,36.0,-41.0,26.0,28.0,7.0,6.0,19.0,23.0,0.0,1.0


The purpose of the below functions is to disambiguate the data. The stats contained in the dataframe are encoded such that for every stat, there is a home and away component. 

For example: The columns 'HS' and 'AS' refer to shots taken by the home team and shots taken by the away team for a given match. This style of encoding is helpful when each row has two teams ('HomeTeam' and 'AwayTeam'), but the next step of processing will see us split every match up, so that each row contains only one team. When this is done, the columns mean different things depending on whether that row's team was home or away for that given match. Every column has to be interpreted with this in mind.

To solve this problem and thus remove the ambiguity, the 'H' and 'A' prefixes in the column names are replaced with 'self' and 'opp'. In this way, every column represents the same stat in relation to the team in question.

It will become clearer what these function are for when I put them to use further below.

In [34]:
def H_to_self_opp(strings, self):
    replaced_strings = []
    for string in strings:
      string = string.replace('AH', '*')
      string = string.replace('WH', '@')
      string = string.replace('HT', '#')
      if self == True:
        string = string.replace('H', '(self)')
        string = string.replace('*', 'AH')
        string = string.replace('@', 'WH')
        string = string.replace('#', 'HT')
        replaced_strings.append(string)
      else:
        string = string.replace('H', '(opp)')
        string = string.replace('*', 'AH')
        string = string.replace('@', 'WH')
        string = string.replace('#', 'HT')
        replaced_strings.append(string)
    return replaced_strings

def A_to_self_opp(strings, self):
    replaced_strings = []
    for string in strings:
      string = string.replace('Avg', '~')
      string = string.replace('AH', '*')
      if self == True:
        string = string.replace('A', '(self)')
        string = string.replace('~', 'Avg')
        string = string.replace('*', 'AH')
        replaced_strings.append(string)
      else:
        string = string.replace('A', '(opp)')
        string = string.replace('~', 'Avg')
        string = string.replace('*', 'AH')
        replaced_strings.append(string)
    return replaced_strings

def disambiguate_home_away(cols, home):
  if home == True:
    new_cols = H_to_self_opp(cols, self=True)
    new_cols = A_to_self_opp(new_cols, self=False)
  else:
    new_cols = H_to_self_opp(cols, self=False)
    new_cols = A_to_self_opp(new_cols, self=True)
  return dict(zip(cols, new_cols))

Here I am processing my dataframe so that there are two rows for every match, one for each team. I'll sort this dataframe and use it to generate team-specific historical data from the past n games played by that team.

I use the functions I just coded to rename the columns accordingly.

In [35]:
df_home_sort = df.sort_values(by=['HomeTeam', 'Date'])
df_away_sort = df.sort_values(by=['AwayTeam', 'Date'])
df_home_sort.drop(columns='AwayTeam', inplace=True)
df_away_sort.drop(columns='HomeTeam', inplace=True)
df_home_sort.rename(columns={'HomeTeam': 'team'}, inplace=True)
df_away_sort.rename(columns={'AwayTeam': 'team'}, inplace=True)
df_home_sort['is_home'] = 1
df_away_sort['is_home'] = 0
home_col_dict = disambiguate_home_away(df_home_sort.columns, home=True)
away_col_dict = disambiguate_home_away(df_away_sort.columns, home=False)
df_home_sort.rename(columns=home_col_dict, inplace=True)
df_away_sort.rename(columns=away_col_dict, inplace=True)
df_sort = pd.concat([df_home_sort, df_away_sort])
df_sort.sort_values(by=['team', 'Date'], inplace=True)
df_sort.reset_index(inplace=True)
df_sort

,index,Div,Date,team,FT(self)G,FT(opp)G,FTR,HT(self)G,HT(opp)G,HTR,...,(self)_rank_2_diff_gd,(opp)_rank_2_abs_diff_points,(opp)_rank_2_diff_gd,(self)_rank_1_abs_diff_points,(self)_rank_1_diff_gd,(opp)_rank_1_abs_diff_points,(opp)_rank_1_diff_gd,is_home,BW(self),IW(self)
0,27932,G1,2019-08-25,AEK,1,2,A,0.0,0.0,D,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1,NaN,NaN
1,27939,G1,2019-09-01,AEK,3,2,A,1.0,1.0,D,...,1.0,0.0,1.0,6.0,-3.0,6.0,-3.0,0,2.35,2.45
2,27944,G1,2019-09-15,AEK,2,0,H,2.0,0.0,H,...,4.0,2.0,4.0,3.0,-2.0,4.0,-2.0,1,NaN,NaN
3,27949,G1,2019-09-21,AEK,1,0,A,0.0,0.0,D,...,6.0,0.0,0.0,3.0,-1.0,9.0,-7.0,0,1.62,1.57
4,27960,G1,2019-09-29,AEK,2,2,D,0.0,1.0,A,...,10.0,10.0,10.0,3.0,1.0,2.0,1.0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56327,12646,N1,2022-04-23,Zwolle,2,0,A,1.0,0.0,A,...,-13.0,16.0,-6.0,38.0,-61.0,33.0,-54.0,0,2.65,2.55
56328,12653,N1,2022-04-30,Zwolle,0,3,H,0.0,1.0,H,...,-10.0,31.0,81.0,38.0,-60.0,11.0,31.0,0,21.00,18.00
56329,12664,N1,2022-05-07,Zwolle,1,1,D,0.0,1.0,A,...,-14.0,2.0,17.0,41.0,-65.0,22.0,-34.0,1,NaN,NaN
56330,12675,N1,2022-05-11,Zwolle,0,2,H,0.0,2.0,H,...,-14.0,21.0,-13.0,44.0,-66.0,42.0,-65.0,0,2.95,2.90


Here I am adding columns to each row which represent the time delta in days from the row's match start time to previous match start times for the last n matches. I'll save these columns to add to my final dataframe later.

In [36]:
# Define the number of columns you want to add (i.e. the number of previous games for which a time delta should be calculated)
n = 7

# Create n new columns, each representing the time delta in days from the current row to the previous n rows
for i in range(1, n+1):
    df_sort[f'delta_{i}'] = (df_sort['Date'] - df_sort['Date'].shift(i)).dt.days

deltas = df_sort[[col for col in df_sort.columns if 'delta' in col]]

deltas

,delta_1,delta_2,delta_3,delta_4,delta_5,delta_6,delta_7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.0,NaN,NaN,NaN,NaN,NaN,NaN
2,14.0,21.0,NaN,NaN,NaN,NaN,NaN
3,6.0,20.0,27.0,NaN,NaN,NaN,NaN
4,8.0,14.0,28.0,35.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
56327,13.0,20.0,35.0,41.0,48.0,56.0,62.0
56328,7.0,20.0,27.0,42.0,48.0,55.0,63.0
56329,7.0,14.0,27.0,34.0,49.0,55.0,62.0
56330,4.0,11.0,18.0,31.0,38.0,53.0,59.0


In [37]:
(df_sort.columns).to_list()

['index',
 'Div',
 'Date',
 'team',
 'FT(self)G',
 'FT(opp)G',
 'FTR',
 'HT(self)G',
 'HT(opp)G',
 'HTR',
 '(self)S',
 '(opp)S',
 '(self)ST',
 '(opp)ST',
 '(self)F',
 '(opp)F',
 '(self)C',
 '(opp)C',
 '(self)Y',
 '(opp)Y',
 '(self)R',
 '(opp)R',
 'B365(self)',
 'B365D',
 'B365(opp)',
 'BWH',
 'BWD',
 'BW(opp)',
 'IWH',
 'IWD',
 'IW(opp)',
 'PS(self)',
 'PSD',
 'PS(opp)',
 'WH(self)',
 'WHD',
 'WH(opp)',
 'VC(self)',
 'VCD',
 'VC(opp)',
 'Max(self)',
 'MaxD',
 'Max(opp)',
 'Avg(self)',
 'AvgD',
 'Avg(opp)',
 'B365>2.5',
 'B365<2.5',
 'P>2.5',
 'P<2.5',
 'Max>2.5',
 'Max<2.5',
 'Avg>2.5',
 'Avg<2.5',
 'AHh',
 'B365AH(self)',
 'B365AH(opp)',
 'PAH(self)',
 'PAH(opp)',
 'MaxAH(self)',
 'MaxAH(opp)',
 'AvgAH(self)',
 'AvgAH(opp)',
 'B365C(self)',
 'B365CD',
 'B365C(opp)',
 'BWC(self)',
 'BWCD',
 'BWC(opp)',
 'IWC(self)',
 'IWCD',
 'IWC(opp)',
 'PSC(self)',
 'PSCD',
 'PSC(opp)',
 'WHC(self)',
 'WHCD',
 'WHC(opp)',
 'VCC(self)',
 'VCCD',
 'VCC(opp)',
 'MaxC(self)',
 'MaxCD',
 'MaxC(opp)',
 'A

Here I am converting the categorical results columns (Home,Draw,Away) to (Win,Draw,Loss) columns in relation to the team in question.

In [38]:
df_sort['win'] = df_sort['FT(self)G'] > df_sort['FT(opp)G']
df_sort['loss'] = df_sort['FT(self)G'] < df_sort['FT(opp)G']
df_sort['draw'] = df_sort['FT(self)G'] == df_sort['FT(opp)G']
df_sort['ht_win'] = df_sort['HT(self)G'] > df_sort['HT(opp)G']
df_sort['ht_loss'] = df_sort['HT(self)G'] < df_sort['HT(opp)G']
df_sort['ht_draw'] = df_sort['HT(self)G'] == df_sort['HT(opp)G']

In [39]:
df_sort

,index,Div,Date,team,FT(self)G,FT(opp)G,FTR,HT(self)G,HT(opp)G,HTR,...,delta_4,delta_5,delta_6,delta_7,win,loss,draw,ht_win,ht_loss,ht_draw
0,27932,G1,2019-08-25,AEK,1,2,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,False,True,False,False,False,True
1,27939,G1,2019-09-01,AEK,3,2,A,1.0,1.0,D,...,NaN,NaN,NaN,NaN,True,False,False,False,False,True
2,27944,G1,2019-09-15,AEK,2,0,H,2.0,0.0,H,...,NaN,NaN,NaN,NaN,True,False,False,True,False,False
3,27949,G1,2019-09-21,AEK,1,0,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,True,False,False,False,False,True
4,27960,G1,2019-09-29,AEK,2,2,D,0.0,1.0,A,...,35.0,NaN,NaN,NaN,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56327,12646,N1,2022-04-23,Zwolle,2,0,A,1.0,0.0,A,...,41.0,48.0,56.0,62.0,True,False,False,True,False,False
56328,12653,N1,2022-04-30,Zwolle,0,3,H,0.0,1.0,H,...,42.0,48.0,55.0,63.0,False,True,False,False,True,False
56329,12664,N1,2022-05-07,Zwolle,1,1,D,0.0,1.0,A,...,34.0,49.0,55.0,62.0,False,False,True,False,True,False
56330,12675,N1,2022-05-11,Zwolle,0,2,H,0.0,2.0,H,...,31.0,38.0,53.0,59.0,False,True,False,False,True,False


This function will add stats from the last n rows as new columns to each row.

In [40]:
def add_last_n_rows(df, n, cols):
    for i in range(1, n + 1):
        temp_df = df.shift(i)
        for col in cols:
            df[f"{col}_last_{i}"] = temp_df[col]
    df = df.reset_index(drop=True)
    return df

Below I use the function. I only am adding the goals scored by both teams and the match result columns for the last 7 games, but any combination of columns could be added using this function. 

Change 'n' to alter the number of previous games for which to add stats, and change 'cols_to_add' to alter which stats are added.

Note: To add all columns from the previous n matches, pass 'df_sort.columns' as the 'cols' parameter.

In [41]:
n = 7
cols_to_add = ['win', 'loss', 'draw', 'FT(self)G', 'FT(opp)G']
df_sort = add_last_n_rows(df=df_sort, n=n, cols=cols_to_add)

In [42]:
df_sort

,index,Div,Date,team,FT(self)G,FT(opp)G,FTR,HT(self)G,HT(opp)G,HTR,...,win_last_6,loss_last_6,draw_last_6,FT(self)G_last_6,FT(opp)G_last_6,win_last_7,loss_last_7,draw_last_7,FT(self)G_last_7,FT(opp)G_last_7
0,27932,G1,2019-08-25,AEK,1,2,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,27939,G1,2019-09-01,AEK,3,2,A,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27944,G1,2019-09-15,AEK,2,0,H,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27949,G1,2019-09-21,AEK,1,0,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27960,G1,2019-09-29,AEK,2,2,D,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56327,12646,N1,2022-04-23,Zwolle,2,0,A,1.0,0.0,A,...,False,True,False,0.0,2.0,False,False,True,1.0,1.0
56328,12653,N1,2022-04-30,Zwolle,0,3,H,0.0,1.0,H,...,True,False,False,1.0,0.0,False,True,False,0.0,2.0
56329,12664,N1,2022-05-07,Zwolle,1,1,D,0.0,1.0,A,...,False,True,False,1.0,2.0,True,False,False,1.0,0.0
56330,12675,N1,2022-05-11,Zwolle,0,2,H,0.0,2.0,H,...,False,True,False,0.0,1.0,False,True,False,1.0,2.0


The above function writes stats from the last n rows regardless of if every stat is really corresponding to the correct team in question. The below function will correct this.

In [43]:
def rolling_window_check(df, column, n):

    # Convert the column to a numeric representation using pd.factorize
    cat_codes, _ = pd.factorize(df[column])
    
    # Create a rolling window of size n over the column
    window = pd.Series(cat_codes).rolling(n)
    
    # Check if all values in the window are equal
    result = window.apply(lambda x: len(set(x)) == 1, raw=True)
    
    # Return a boolean series indicating if the condition is True or False for each row
    return result.fillna(False)


In [44]:
df_sort['check'] = rolling_window_check(df=df_sort, column='team', n=n)

Now dropping the columns which do not have adequate historical data.

In [45]:
df_sort = df_sort[df_sort['check'] != False]
df_sort.drop(columns=['check'], axis=1, inplace=True)
df_sort

,index,Div,Date,team,FT(self)G,FT(opp)G,FTR,HT(self)G,HT(opp)G,HTR,...,win_last_6,loss_last_6,draw_last_6,FT(self)G_last_6,FT(opp)G_last_6,win_last_7,loss_last_7,draw_last_7,FT(self)G_last_7,FT(opp)G_last_7
6,27973,G1,2019-10-20,AEK,3,2,H,1.0,1.0,D,...,False,True,False,1.0,2.0,NaN,NaN,NaN,NaN,NaN
7,27980,G1,2019-10-27,AEK,0,2,H,0.0,1.0,H,...,True,False,False,3.0,2.0,False,True,False,1.0,2.0
8,27986,G1,2019-11-03,AEK,3,2,H,0.0,1.0,A,...,True,False,False,2.0,0.0,True,False,False,3.0,2.0
9,27994,G1,2019-11-10,AEK,2,3,H,2.0,0.0,A,...,True,False,False,1.0,0.0,True,False,False,2.0,0.0
10,28001,G1,2019-11-24,AEK,1,1,D,0.0,0.0,D,...,False,False,True,2.0,2.0,True,False,False,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56327,12646,N1,2022-04-23,Zwolle,2,0,A,1.0,0.0,A,...,False,True,False,0.0,2.0,False,False,True,1.0,1.0
56328,12653,N1,2022-04-30,Zwolle,0,3,H,0.0,1.0,H,...,True,False,False,1.0,0.0,False,True,False,0.0,2.0
56329,12664,N1,2022-05-07,Zwolle,1,1,D,0.0,1.0,A,...,False,True,False,1.0,2.0,True,False,False,1.0,0.0
56330,12675,N1,2022-05-11,Zwolle,0,2,H,0.0,2.0,H,...,False,True,False,0.0,1.0,False,True,False,1.0,2.0


Here I am preparing the historical data to be added to the main dataframe. The rows were previously split so that each contained only one team in order to generate the historical data. Now the rows must be rebuilt to represent matches between two teams again. I make sure to remove the columns that represent the current match in each row. I will add current stats when I'm assembling the final data.

In [46]:
indices = df_sort['index']
df_sort = df_sort.drop(columns=['team'], axis=1)
df_sort = df_sort[[col for col in df_sort.columns if 'index' not in col]]
df_sort['index'] = indices
df_hist_h = df_sort[df_sort['is_home'] == 1]
df_hist_a = df_sort[df_sort['is_home'] == 0]
df_hist_h = df_hist_h.add_prefix('home_')
df_hist_a = df_hist_a.add_prefix('away_')
df_hist_h = df_hist_h.rename(columns={'home_index': 'index'})
df_hist_a = df_hist_a.rename(columns={'away_index': 'index'})
df_hist = pd.merge(df_hist_h, df_hist_a, on='index')
indices = df_hist['index']
col_list = [col for col in df_hist.columns if 'last' in col or 'delta' in col]
df_hist = df_hist[col_list]
df_hist['index'] = indices

Here's the historical data ready to be added to the original dataframe.

In [47]:
df_hist

,home_delta_1,home_delta_2,home_delta_3,home_delta_4,home_delta_5,home_delta_6,home_delta_7,home_win_last_1,home_loss_last_1,home_draw_last_1,...,away_loss_last_6,away_draw_last_6,away_FT(self)G_last_6,away_FT(opp)G_last_6,away_win_last_7,away_loss_last_7,away_draw_last_7,away_FT(self)G_last_7,away_FT(opp)G_last_7,index
0,15.0,21.0,29.0,35.0,49.0,56.0,NaN,False,False,True,...,False,False,2.0,1.0,False,True,False,1.0,2.0,27973
1,7.0,14.0,29.0,35.0,43.0,49.0,63.0,False,True,False,...,True,False,2.0,3.0,False,False,True,2.0,2.0,27986
2,14.0,21.0,28.0,35.0,50.0,56.0,64.0,False,True,False,...,True,False,2.0,3.0,False,False,True,2.0,2.0,28001
3,7.0,13.0,27.0,34.0,41.0,48.0,63.0,False,True,False,...,True,False,0.0,1.0,False,True,False,1.0,4.0,28011
4,4.0,11.0,18.0,24.0,38.0,45.0,52.0,True,False,False,...,False,True,1.0,1.0,True,False,False,4.0,1.0,28027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26535,7.0,15.0,21.0,29.0,36.0,50.0,58.0,True,False,False,...,False,False,4.0,1.0,False,True,False,0.0,1.0,12612
26536,15.0,21.0,28.0,36.0,42.0,50.0,57.0,False,True,False,...,True,False,1.0,4.0,False,True,False,1.0,2.0,12628
26537,7.0,22.0,28.0,35.0,43.0,49.0,57.0,False,True,False,...,False,False,2.0,1.0,True,False,False,4.0,1.0,12640
26538,7.0,14.0,27.0,34.0,49.0,55.0,62.0,False,True,False,...,True,False,0.0,1.0,False,False,True,1.0,1.0,12664


Now I am assembling the final data.

Below, I add the historical data to the original dataframe.

In [48]:
data = df.merge(df_hist, left_index=True, right_on='index')
data = data.drop(columns='index')

In [49]:
data

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,away_win_last_6,away_loss_last_6,away_draw_last_6,away_FT(self)G_last_6,away_FT(opp)G_last_6,away_win_last_7,away_loss_last_7,away_draw_last_7,away_FT(self)G_last_7,away_FT(opp)G_last_7
7021,E0,2022-08-05,Crystal Palace,Arsenal,0,2,A,0.0,1.0,A,...,True,False,False,3.0,1.0,True,False,False,4.0,2.0
9761,E0,2022-08-06,Fulham,Liverpool,2,2,D,1.0,0.0,H,...,True,False,False,2.0,0.0,True,False,False,4.0,0.0
3977,E0,2022-08-06,Bournemouth,Aston Villa,2,0,H,1.0,0.0,H,...,True,False,False,2.0,0.0,False,False,True,0.0,0.0
13517,E0,2022-08-06,Leeds,Wolves,2,1,H,1.0,1.0,D,...,False,True,False,0.0,1.0,False,True,False,0.0,1.0
16514,E0,2022-08-06,Newcastle,Nott'm Forest,2,0,H,0.0,0.0,D,...,True,False,False,4.0,0.0,False,True,False,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13154,G1,2020-07-18,Larisa,Xanthi,0,0,D,0.0,0.0,D,...,False,True,False,0.0,1.0,False,False,True,1.0,1.0
18098,G1,2020-07-18,Panetolikos,Volos NFC,1,0,H,1.0,0.0,H,...,False,True,False,0.0,1.0,False,True,False,1.0,4.0
17365,G1,2020-07-19,Olympiakos,AEK,3,0,H,2.0,0.0,H,...,False,True,False,1.0,2.0,False,False,True,2.0,2.0
18038,G1,2020-07-19,Panathinaikos,OFI Crete,3,2,H,3.0,2.0,H,...,False,False,True,0.0,0.0,False,True,False,1.0,3.0


Here is code that will help get a sense of which columns have nan values. We will need to remove the rows which contain them.

In [50]:
nan_cols = data.columns[data.isna().any()].tolist()

# Print the names of the columns with NaN values and their counts
for col in nan_cols:
    print(col, data[col].isna().sum())

HTHG 4
HTAG 4
HTR 4
HS 1765
AS 1765
HST 1765
AST 1765
HF 1767
AF 1767
HC 1765
AC 1765
HY 4
AY 4
HR 4
AR 4
B365H 70
B365D 70
B365A 70
BWH 664
BWD 664
BWA 664
IWH 106
IWD 106
IWA 106
PSH 130
PSD 130
PSA 130
WHH 688
WHD 688
WHA 688
VCH 176
VCD 176
VCA 176
MaxH 15
MaxD 15
MaxA 15
AvgH 15
AvgD 15
AvgA 15
B365>2.5 83
B365<2.5 83
P>2.5 156
P<2.5 156
Max>2.5 16
Max<2.5 16
Avg>2.5 16
Avg<2.5 16
AHh 17
B365AHH 239
B365AHA 240
PAHH 132
PAHA 131
MaxAHH 17
MaxAHA 17
AvgAHH 17
AvgAHA 17
B365CH 32
B365CD 32
B365CA 32
BWCH 482
BWCD 482
BWCA 482
IWCH 80
IWCD 80
IWCA 80
PSCH 23
PSCD 23
PSCA 23
WHCH 652
WHCD 652
WHCA 652
VCCH 33
VCCD 33
VCCA 33
MaxCH 2
MaxCD 2
MaxCA 2
AvgCH 2
AvgCD 2
AvgCA 2
B365C>2.5 36
B365C<2.5 36
PC>2.5 46
PC<2.5 46
MaxC>2.5 2
MaxC<2.5 2
AvgC>2.5 2
AvgC<2.5 2
AHCh 2
B365CAHH 60
B365CAHA 60
PCAHH 24
PCAHA 24
MaxCAHH 2
MaxCAHA 2
AvgCAHH 2
AvgCAHA 2
home_delta_7 1
home_win_last_7 1
home_loss_last_7 1
home_draw_last_7 1
home_FT(self)G_last_7 1
home_FT(opp)G_last_7 1


Here, I'll drop any rows containing NaN values. The data still needs a tiny bit of processing before it's ready to go (one-hot encoding, creation of targets, etc.) You can find examples of this in the 'demo' notebook.

In [51]:
data = data.dropna()
data

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,away_win_last_6,away_loss_last_6,away_draw_last_6,away_FT(self)G_last_6,away_FT(opp)G_last_6,away_win_last_7,away_loss_last_7,away_draw_last_7,away_FT(self)G_last_7,away_FT(opp)G_last_7
7021,E0,2022-08-05,Crystal Palace,Arsenal,0,2,A,0.0,1.0,A,...,True,False,False,3.0,1.0,True,False,False,4.0,2.0
9761,E0,2022-08-06,Fulham,Liverpool,2,2,D,1.0,0.0,H,...,True,False,False,2.0,0.0,True,False,False,4.0,0.0
3977,E0,2022-08-06,Bournemouth,Aston Villa,2,0,H,1.0,0.0,H,...,True,False,False,2.0,0.0,False,False,True,0.0,0.0
13517,E0,2022-08-06,Leeds,Wolves,2,1,H,1.0,1.0,D,...,False,True,False,0.0,1.0,False,True,False,0.0,1.0
16514,E0,2022-08-06,Newcastle,Nott'm Forest,2,0,H,0.0,0.0,D,...,True,False,False,4.0,0.0,False,True,False,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25091,G1,2020-07-11,Volos NFC,Atromitos,2,3,A,0.0,3.0,A,...,True,False,False,2.0,1.0,True,False,False,1.0,0.0
26280,G1,2020-07-11,Xanthi,Asteras Tripolis,1,2,A,0.0,1.0,A,...,False,True,False,0.0,3.0,False,False,True,1.0,1.0
17225,G1,2020-07-11,OFI Crete,Aris,0,1,A,0.0,1.0,A,...,False,True,False,1.0,3.0,True,False,False,3.0,1.0
18037,G1,2020-07-12,Panathinaikos,AEK,1,3,A,1.0,2.0,A,...,True,False,False,2.0,0.0,False,False,True,1.0,1.0


You can save the processed data to your Google Drive using this code.

In [52]:
import os
from google.colab import drive
os.chdir('/content/drive/MyDrive/WorkingFolder')
data.to_csv('processed-data.csv', index=False)